# 5.2 Potenciales y campos cerca de cargas eléctricas

En regiones donde sí hay carga, se cumple la ecuación de Poisson:

$\nabla^2 V = - \dfrac{\rho}{\epsilon_0}$

si añadimos el término de la densidad de carga a la diferencias de la ecuación de Laplace tenemos:

$V(i, j, k)= \dfrac{1}{6} \left[ V(i+1, j, k) + V(i -1, j,k) + V(i, j +1, k) + V(i, j-1, k) + V(i, j, k+1,) V(i, j, k-1)  \right] + \dfrac{\rho(i,j,k) (\Delta x)^2 }{6 \epsilon_0}  $ 

este resultado está hecho bajo la suposición de que la división de la rejilla es la misma en las 3 dimensiones ($\Delta x = \Delta y = \Delta z$). 

Los métodos de relajación como Jacobi, Gauss-Seidel o la sobrerrelajación sirve para estos fines

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numerical import *
%matplotlib widget

In [ ]:
def SOR_Poisson2D(V, indicesCC,rho,  err=1e-5, imax=1000):
    iteracion=0; errores=[]  # Inicializamos
    V_next = V.copy()
    ny, nx= np.shape(V)
    alpha = 1
    
    def NeighbourAverage(V, i, j):
        a = (V[i+1, j] + V[i-1, j] + V[i, j+1] + V[i, j-1])/4.
        return a
        

    #Este ciclo while realiza el método de sobrerrejación
    while True:
        
        # Los límites de los for se les quitan las 'orillas' para que cada punto tenga vecinos.
        for i in range(1, nx-1):
            for j in range(1, ny-1):
                #Filtramos los puntos no definidos por las CC
                if (i, j) in indicesCC:
                    pass
                else: 
                    #Solución por Gauss-Seidel
                    #print(rho(i, j))
                    tmp = V_next[i, j]   #Variable temporal para evitar mmdas
                    V_GS = NeighbourAverage(V_next, i, j) +  rho(i, j)/4.  #Gauss-Seidel
                    ajuste = alpha*(V_GS - tmp) #Ajuste del método SOR
                    V_next[i, j] = ajuste + tmp  #SOR
                    #Se guarda el error por punto
                    error =abs(1 - V_GS/V_next[i,j])
                    errores.append(error)

        #El error será el máximo de los errores de cada punto
        errmax = max(errores)
        iteracion += 1        
        if errmax < err: #¿La diferencia de potencial es menor al mínimo establecido?
            print("Tolerancia de error mínima lograda. Err:", errmax)
            print("Iteraciones: ", iteracion)
            break

        elif iteracion >= imax: #¿Se alcanzó el límite de iteraciones?
            print("Límite de iteraciones alcanzado:", iteracion)
            break

        else:
            #Reiniciamos
            errores=[]
        
    return V_next, iteracion, errmax

In [ ]:
def Discretiza3D(X, Y, Z,  N=[10, 10, 10]):
    """Discretiza el plano en la región x[x0, x1], Y en [y0, y1] en secciones
        N = [nx, ny]"""
    
    x = np.linspace(X[0], X[1], N[0])
    y = np.linspace(Y[0], Y[1], N[1])
    z = np.linspace(Z[0], Z[1], N[2])
    V = np.zeros([N[0], N[1], N[2]])
    
    return  V, x

In [ ]:
X = [-1, 1]
Y = [-1, 1]
nx = ny = 51
N = [nx, ny]

X, Y, V= Discretiza(X, Y, N)

np.shape(V)

In [ ]:
ix =int(nx/2); iy = int(ny/2) 

In [ ]:
def rho(i, j):
    return 1 if (i, j)==(25, 25) else 0

In [ ]:
def CC2D(V):
    nx = np.shape(V)[0]
    indicesCC = []
    
    for i in range(nx):
        V[i, 0] = 0;
        indicesCC.append((i, 0))
        V[0, i] = 0;
        indicesCC.append((0, i))
        
        V[i, nx-1] = 0;
        indicesCC.append((i, nx-1))
        V[nx-1, i] = 0;
        indicesCC.append((nx-1, i))
    
    return V, indicesCC
        
        
        

In [ ]:
V_bou, indicesCC = CC2D(V)
V_bou

In [ ]:
V_out, iteracion, diff = SOR_Poisson2D(V_bou, indicesCC, rho)


In [ ]:

Surface3D(X, Y, V_out)

In [ ]:
from scipy.integrate import solve_bvp

La idea es convertir la ec. de Poisson en un sistema de ecuacioes de primer orden:

$ \dfrac{\partial^2 V}{\partial x^2} + \dfrac{\partial^2 V}{\partial y^2 + \dfrac{\partial^2 V}{\partial z^2 $